In [ ]:
!apt-get install openslide-tools
!pip install openslide-python
!pip install tifffile

In [ ]:
#ASAP Program to convert xml in tif images (multiresolutionimageinterface) suggested by Camelyon challenge
#The shared XML files are compatible with the [ASAP](https://github.com/GeertLitjens/ASAP) software. 

#!lsb_release -a # cheking linux

!wget https://github.com/computationalpathologygroup/ASAP/releases/download/1.9/ASAP-1.9-Linux-Ubuntu1804.deb

!apt-get install "./ASAP-1.9-Linux-Ubuntu1804.deb"

!dpkg -L asap

import sys
sys.path.insert(0,"/opt/ASAP/bin")

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',  force_remount=True)

In [ ]:
import multiresolutionimageinterface as mir
import matplotlib.pyplot as plt
import os.path as osp
import openslide
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import numpy as np

In [ ]:
slide_num='102'

slide_path = '/content/drive/My Drive/tumor/tumor_' + slide_num + '.tif'
anno_path = '/content/drive/My Drive/cancer_detection/lesion_annotations/tumor_' + slide_num + '.xml'
mask_path = '/content/drive/My Drive/cancer_detection/annotation_mask/teste2/'

In [ ]:
tumor_paths = glob.glob(slide_path)
tumor_paths.sort()
anno_tumor_paths = glob.glob(anno_path)
anno_tumor_paths.sort()

reader = mir.MultiResolutionImageReader()

mr_image = reader.open(slide_path)
annotation_list=mir.AnnotationList()
xml_repository = mir.XmlRepository(annotation_list)
xml_repository.setSource(anno_path)
xml_repository.load()
annotation_mask=mir.AnnotationToMask()
camelyon17_type_mask = False
label_map = {'metastases': 1, 'normal': 2} if camelyon17_type_mask else {'_0': 255, '_1': 255, '_2': 0}
conversion_order = ['metastases', 'normal'] if camelyon17_type_mask else  ['_0', '_1', '_2']
output_path= mask_path + 'tumor_' + slide_num + '_mask.tif'
annotation_mask.convert(annotation_list, output_path, mr_image.getDimensions(), mr_image.getSpacing(), label_map, conversion_order)

In [ ]:
from openslide import OpenSlide
from openslide import open_slide

def read_slide(slide, x, y, level, width, height, as_float=False):
    im = slide.read_region((x,y), level, (width, height))
    im = im.convert('RGB') # drop the alpha channel
    if as_float:
        im = np.asarray(im, dtype=np.float32)
    else:
        im = np.asarray(im)
    assert im.shape == (height, width, 3)
    return im


slide_path = "/content/drive/My Drive/tumor/tumor_047.tif"
tumor_mask_path = "/content/drive/My Drive/cancer_detection/annotation_mask/teste2/tumor_047_mask.tif"

slide = open_slide(slide_path)
tumor_mask = open_slide(tumor_mask_path)

slide_image = read_slide(slide, 
                         x=0, 
                         y=0, 
                         level=5, 
                         width=slide.level_dimensions[5][0], 
                         height=slide.level_dimensions[5][1])

mask_image = read_slide(tumor_mask, 
                         x=0, 
                         y=0, 
                         level=5, 
                         width=slide.level_dimensions[5][0], 
                         height=slide.level_dimensions[5][1])
#show figs
plt.figure(figsize=(10,10), dpi=100)
plt.imshow(slide_image)

plt.figure(figsize=(10,10), dpi=100)
plt.imshow(mask_image)
